In [3]:
from __future__ import unicode_literals, print_function
import plac #  wrapper over argparse
import random
from pathlib import Path
import spacy
import pandas as pd
import numpy as np

In [6]:
!git clone https://github.com/Lednik7/SpacyToolKit.git
!python -m pip install -r SpacyToolKit/requirements.txt

Cloning into 'SpacyToolKit'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 107 (delta 55), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (107/107), 31.46 KiB | 441.00 KiB/s, done.
Resolving deltas: 100% (55/55), done.


In [18]:
from SpacyToolKit.formatting.preprocessing_v2 import *

In [24]:
train = pd.read_csv("train.csv")

In [22]:
desc = train.description.to_list()
skill = train.key_skills.to_list()

In [23]:
#предобработаем данные
TRAIN_DATA = preprocessing(desc, skill, "Skills")
TRAIN_DATA = choice(delete_None(TRAIN_DATA))

100%|██████████| 17954/17954 [00:09<00:00, 1879.22it/s]


In [20]:
def main():
    model = ("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir = (".", "option", "o", Path),
    n_iter = (1, "option", "n", int)
plac.call(main)

usage: ipykernel_launcher.py [-h]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-472ec05b-7aaf-4cf4-90f6-651f33accf5b.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#создадим модель и её pipe
nlp = spacy.load("en")
n_iter=1

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

In [ ]:
#отберем те данные, что пригодны для модели, а также примерно расчитаем время выполнения одной итерации
basket = []
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(1):
        print("\n", itn + 1)
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
          try:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
            
            basket.append((text, annotations))
          except:
            pass
        print(losses)

  0%|          | 1/3065 [00:00<06:13,  8.20it/s]

1


100%|██████████| 3065/3065 [05:35<00:00,  9.14it/s]

{'ner': 25141.63619172928}


In [ ]:
len(basket)

2994

In [ ]:
#сбросим веса
nlp = spacy.load("en")
n_iter=5

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

In [ ]:
TRAIN_DATA = basket

In [ ]:
# Обучаем модель
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA, mininterval=10):
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
            
        
        print(losses)

  0%|          | 0/2994 [00:00<?, ?it/s]

{'ner': 25193.94714230446}


  0%|          | 0/2994 [00:00<?, ?it/s]

{'ner': 21069.44967105342}


  0%|          | 0/2994 [00:00<?, ?it/s]

{'ner': 20339.325111051978}


  0%|          | 0/2994 [00:00<?, ?it/s]

{'ner': 19586.341195105095}


100%|██████████| 2994/2994 [05:30<00:00,  9.06it/s]

{'ner': 19106.9390943734}


In [ ]:
def show(train):
  a = []
  for i in train[1]["entities"]:
    a.append(train[0][i[0]:i[1]])
  return a

In [ ]:
#тестируем модель
from sklearn.metrics import accuracy_score
for text, b in TRAIN_DATA[:100]:
    doc = nlp(text)
    print('Entities', [(ent.text) for ent in doc.ents], " | ", show((text, b)))

Entities ['iOS', 'iOS', 'Objective-C', 'iOS', 'OpenGL']  |  ['iOS', 'iOS', 'iOS', 'Objective-C']
Entities ['PHP7', 'Javascript', 'jQuery', 'MySQL', 'Redis', 'Elasticsearch', 'Linux', 'PHP', 'Git', 'Git', 'Javascript', 'jQuery', 'MySQL', 'Redis', 'Elasticsearch', 'Linux', 'PHP', 'Git', 'Git']  |  ['Git', 'Redis', 'Javascript', 'Javascript', 'MySQL', 'Redis', 'PHP', 'Git', 'MySQL', 'PHP', 'Git', 'Git', 'PHP', 'PHP']
Entities ['JavaScript', 'Node.js', 'Big Data', 'Google Material', 'PostgreSQL', 'MongoDB', 'JavaScript', 'JavaScript']  |  ['Node.js', 'JavaScript', 'JavaScript', 'JavaScript', 'PostgreSQL', 'MongoDB']
Entities ['Google Adwords', 'Google Analytics', 'Google Analytics']  |  ['Google Analytics', 'Google Analytics', 'Google Adwords']
Entities ['Коммуникабельность', 'грамотная речь']  |  ['маникюр', 'маникюр', 'педикюр', 'педикюр']
Entities ['SOA']  |  ['AWS Elastic Beanstalk', 'CloudFormation', 'Amazon Web Services', 'Docker', 'Docker', 'Docker', 'remote work', 'Crossover', 'Doc

In [ ]:
#сохраняем модель
nlp.to_disc("path")